In [229]:
import json, csv
from itertools import permutations

def calculate_lineup_shapley(player_names):
    try:
        stat_name = 'NET_RATING'
        names_to_ids = {}
        ids_to_names = {}
        for p in player_names:
            with open('players.json', 'r') as f:
                d = json.loads(f.read())
                first_name = p.split(' ')[0]
                last_name = p.split(' ')[-1]
                for el in d:
                    if el['firstName']==first_name and el['lastName']==last_name:
                        names_to_ids[p] = str(el['playerId'])
                        ids_to_names[str(el['playerId'])] = p
        d = {}
        with open(f'player_{stat_name}.csv', 'r') as f:
            reader = csv.DictReader(f)
            for row in reader:
                if str(row['player_id']) in names_to_ids.values():
                    d[frozenset([row['player']])]=float(row[stat_name])
        with open(f'lineups_{stat_name}.csv', 'r') as f:
            reader = csv.DictReader(f)
            for row in reader:
                if set(row['lineup_ids'].split('-'))<=set(names_to_ids.values()):
                    s = set()
                    for el in row['lineup_ids'].split('-'):
                        s.add(ids_to_names[el])
                    d[frozenset(s)] = float(row[stat_name])

        delta = []
        for p in permutations(player_names):
            delta.append({})
            s = set()
            t = 0
            for el in p:
                s.add(el)
                key = frozenset(s)
                delta[-1][el] = d[key] - t
                t = d[key]

        shapley = {}
        for p in player_names:
            vals = [d[p] for d in delta]
            shapley[p] = sum(vals)/len(vals)
        return shapley
    except:
        return None

In [230]:
lineup_shapleys = []
with open('test_lineups.txt', 'r') as f:
    for line in f.readlines():
        lineup = line.rstrip().split(';')
        values = calculate_lineup_shapley(lineup)
        if values is not None:
            lineup_shapleys.append(values)
        print(line)

Dewayne Dedmon;Taurean Prince;John Collins;Kevin Huerter;Trae Young

Kent Bazemore;Dewayne Dedmon;John Collins;Kevin Huerter;Trae Young

Dewayne Dedmon;DeAndre' Bembry;John Collins;Kevin Huerter;Trae Young

Al Horford;Kyrie Irving;Marcus Morris;Marcus Smart;Jayson Tatum

Al Horford;Gordon Hayward;Kyrie Irving;Jaylen Brown;Jayson Tatum

Jared Dudley;Joe Harris;D'Angelo Russell;Caris LeVert;Jarrett Allen

Joe Harris;D'Angelo Russell;Treveon Graham;Jarrett Allen;Rodions Kurucs

Joe Harris;D'Angelo Russell;Rondae Hollis-Jefferson;Jarrett Allen;Rodions Kurucs

Marvin Williams;Nicolas Batum;Kemba Walker;Jeremy Lamb;Cody Zeller

Marvin Williams;Nicolas Batum;Bismack Biyombo;Kemba Walker;Jeremy Lamb

Justin Holiday;Zach LaVine;Jabari Parker;Ryan Arcidiacono;Wendell Carter Jr.

Robin Lopez;Otto Porter Jr.;Zach LaVine;Kris Dunn;Lauri Markkanen

Justin Holiday;Kris Dunn;Ryan Arcidiacono;Lauri Markkanen;Wendell Carter Jr.

Justin Holiday;Zach LaVine;Jabari Parker;Cameron Payne;Wendell Carter Jr.



Trevor Ariza;Jeff Green;Bradley Beal;Tomas Satoransky;Bobby Portis

Bradley Beal;Tomas Satoransky;Bobby Portis;Thomas Bryant;Troy Brown Jr.

Dwight Howard;John Wall;Markieff Morris;Bradley Beal;Otto Porter Jr.

Trevor Ariza;Jeff Green;Bradley Beal;Tomas Satoransky;Otto Porter Jr.



In [231]:
from bokeh.io import output_file, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Label, Title
from bokeh.models.glyphs import ImageURL
import requests, shutil


image_urls = {}
with open('players.json', 'r') as f:
    data = json.loads(f.read())
    for el in data:
        image_urls[el['firstName']+' '+el['lastName']] = f"https://stats.nba.com/media/players/230x185/{el['playerId']}.png"

def draw_shapley_chart(shapleys):
    output_notebook()
    data = sorted(shapleys.items(), key=lambda kv: kv[1])[::-1]
    names = [el[0] for el in data]
    print(data)
    bottoms = []
    tops = []
    colors = []
    v = 0
    for el in data:
        bottoms.append(v)
        if el[1]<0:
            colors.append('firebrick')
        else:
            colors.append('green')
        v = el[1]+v
        tops.append(v)
    print(tops, bottoms)
    min_y = min(tops)
    if min_y>min(bottoms):
        min_y = min(bottoms)
    max_y = max(tops)
    if max_y > max(tops):
        max_y = max(tops)
    p = figure(x_range=names, y_range=[min_y-5, max_y+5], plot_width=500, plot_height=500, title="Shapley Contributions to a Lineup's Net Rating")              
    
    p.add_layout(Title(text=f"Lineup: {','.join([x.split(' ')[-1] for x in names])}", text_font_style="italic"), 'above')
    p.add_layout(Title(text=f"Net Rating: {str(tops[-1])[:5]}", text_font_style="italic"), 'above')
    p.xaxis.major_label_orientation = 0.6
    p.xaxis.axis_label = 'Player'
    p.yaxis.axis_label = 'Net Rating Per 100 Possessions'
    x = [0.5, 1.5, 2.5, 3.5, 4.5]
    p.vbar(x=x, width=0.5, bottom=bottoms,
       top=tops, color=colors)
    
    urls = [image_urls[i] for i in names]   
    p.image_url(url=urls, x=[i-0.3 for i in x], y=bottoms, w=0.6, h=4.2)
    for i in range(0, len(x)):
        if colors[i]=='green':
            y = tops[i]
        else:
            y = tops[i]-0.12
        l = Label(x=x[i]-0.2, y=y, text=str(tops[i])[:6], text_font_size='12px', text_color='black')
        p.add_layout(l)
        
    for i in range(0, len(x)):
        y = float(bottoms[i]+tops[i])/2
        l = Label(x=x[i]-0.2, y=y-0.1, text=str(data[i][1])[:6], text_font_size='12px', text_color='gold')
        p.add_layout(l)
    show(p)

In [233]:
sorted_lineup_shapleys = list(filter(lambda x: len(x)==5, sorted(lineup_shapleys, key=lambda x: sum(x.values()), reverse=False)))
for lineup in sorted_lineup_shapleys[:1]:
    draw_shapley_chart(lineup)

Loading BokehJS ...

[('Mikal Bridges', 0.17999999999999988), ('Deandre Ayton', -1.2533333333333327), ('T.J. Warren', -2.903333333333335), ('Devin Booker', -4.569999999999995), ("De'Anthony Melton", -7.6533333333333315)]
[0.17999999999999988, -1.0733333333333328, -3.976666666666668, -8.546666666666663, -16.199999999999996] [0, 0.17999999999999988, -1.0733333333333328, -3.976666666666668, -8.546666666666663]
